In [3]:
import torch

embedding = torch.rand(10, 56,100,100)

In [1]:
# let's get the client and server model from the embedding
from Models.mobilenetv2 import MobileNetV2
from Dataloaders.dataloader_cifar10 import Dataloader_cifar10
import torch.nn as nn
import torch.optim as optim
import torch
from tqdm import tqdm
import os
import sys
import cv2

model = MobileNetV2(num_classes=10)
client_model = model.get_client_model()
server_model = model.get_server_model()

# split the server model into 2 parts
server_model_1 = server_model[:len(server_model)-1]
server_model_2 = server_model[-1]

# client model and server 1 are fixed, server 2 is trainable
client_model = client_model.train()
server_model_1 = server_model_1.train()
server_model_2 = server_model_2.train()

loss = torch.nn.CrossEntropyLoss()
# https://github.com/d-li14/mobilenetv2.pytorch
# batch size 256; epoch 150; learning rate 0.05; LR decay strategy cosine; weight decay 0.00004
optimizer = torch.optim.Adam(server_model_2.parameters(), lr=0.05)

# put them in cuda
client_model = client_model.cuda()
server_model_1 = server_model_1.cuda()
server_model_2 = server_model_2.cuda()

# get the data
train, test, val, labels = Dataloader_cifar10(128, 2014)

# train
epoch = 20
for i in tqdm(range(epoch)):
    for data in train:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        # with torch.no_grad():
        outputs = client_model(inputs)
        outputs = server_model_1(outputs)
        outputs = server_model_2(outputs)
        # get the index of the max value of the output?
        loss_train = loss(outputs, labels)
        loss_train.backward()
        optimizer.step()
    print('train loss: ',loss_train.item())
    
    # val
    if i % 5 == 0:
        for data in val:
            inputs, labels = data
            inputs = inputs.cuda()
            labels = labels.cuda()
            with torch.no_grad():
                outputs = client_model(inputs)
                outputs = server_model_1(outputs)
                outputs = server_model_2(outputs)
            loss_val = loss(outputs, labels)
        print('val loss: ', loss_val.item())

# test
correct = 0
total = 0
with torch.no_grad():
    for data in test:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = client_model(inputs)
        outputs = server_model_1(outputs)
        outputs = server_model_2(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total)) 


 

/home/tonypeng/anaconda3/envs/iot/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/20 [00:00<?, ?it/s]

train loss:  1.272678256034851


  5%|▌         | 1/20 [00:33<10:38, 33.58s/it]

...

 95%|█████████▌| 19/20 [09:37<00:29, 29.71s/it]

train loss:  2.799011468887329


100%|██████████| 20/20 [10:07<00:00, 30.38s/it]

train loss:  2.5260791778564453


Accuracy of the network on the 10000 test images: 72 %


In [3]:
import importlib
from Models.mobilenetv2 import MobileNetV2
from Dataloaders.dataloader_cifar10 import Dataloader_cifar10
importlib.reload(MobileNetV2)
importlib.reload(Dataloader_cifar10)
model = MobileNetV2(num_classes=10, weight_path='./otherwork/pytorch-cifar/savedpth/cifar10-mobV2-200.pth')


ImportError: module MobileNetV2 not in sys.modules